In [53]:
import pandas as pd
import numpy as np

In [54]:
#import prices and sales dataset
prices_df = pd.read_csv('prices.csv')
sales_df = pd.read_csv('sales.csv')

In [55]:
#check top 5 of prices df
prices_df.head()

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32


In [56]:
#check top 5 of sales df
sales_df.head()

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1


In [57]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB


In [58]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  14 non-null     int64 
 1   old_price   14 non-null     int64 
 2   new_price   14 non-null     int64 
 3   updated_at  14 non-null     object
dtypes: int64(3), object(1)
memory usage: 576.0+ bytes


In [59]:
#convert 'updated_at' & 'ordered_at' to datetime
prices_df['updated_at'] = pd.to_datetime(prices_df['updated_at'])
sales_df['ordered_at'] = pd.to_datetime(sales_df['ordered_at'])

In [60]:
#sort by date
prices_df = prices_df.sort_values('updated_at')
sales_df = sales_df.sort_values('ordered_at')

In [61]:
#merge prices and sales dataframes using merge_asof with direction: nearest
merged_df = pd.merge_asof(sales_df, prices_df, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')

In [62]:
#check the top 5
merged_df.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00


In [63]:
#choosing price bw 'old_price' & 'new_price'
merged_df['price'] = np.where(merged_df['ordered_at'] >= merged_df['updated_at'], merged_df['new_price'], merged_df['old_price'])

In [64]:
#calculate revenue
merged_df['revenue'] = merged_df['new_price'] * merged_df['quantity_ordered']

In [65]:
merged_df.head()

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,price,revenue
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000,53500
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000,53500
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000,53500
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500,53500


In [66]:
#summarize revenue of each product_id
result = merged_df.groupby(['product_id', 'price'])['revenue'].sum().reset_index()

In [67]:
result

,product_id,price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,188000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,31000
7,4085861,52000,1040000
8,4085861,53500,2204500
9,4085861,58000,2118500
